### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import logging
import datetime, time
import platform
import itertools
# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt

# OpenTNSim
import opentnsim

# Used for mathematical functions
import math             
import tqdm
# Used for making the graph to visualize our problem
import networkx as nx   

import plotly.express as px
from plotly.subplots import make_subplots

#logging.basicConfig(level=logging.DEBUG) #turn on all the debug messages
logging.basicConfig(level=logging.INFO)# turn off all the debug messages


### Create vessel - add VesselProperties and ConsumesEnergy mixins


In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel_up = {
    "env": None,
    "name": 'Vessel',
    "route": None,
    "geometry": None,
    "V_g_ave": None,  # m/s
    "type": None,
    "B": 10.5,
    "L": 155,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": True, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "V_g_profile":None, # if use the V_g_profile to determine V_w, set to True, otherwise (use the V_g_ave to determine V_w) set to False.
    "P_tot_given_profile":False,
    "P_installed": 780, # kW  
    "P_tot_given": True, # kW
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": True,
    "sailing_upstream":False,
    "wind_influence": False, # if consider wind influence, set to True; otherwise set to False.
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.9, 
    "C_year": 1961,
}             


In [4]:
# Create a dict with all important settings

data_vessel_down = {
    "env": None,
    "name": 'Vessel',
    "route": None,
    "geometry": None,
    "V_g_ave": None,  # m/s
    "type": None,
    "B": 20,
    "L": 85,
    "H_e": None, 
    "H_f": None, 
    "T": 1,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": True, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "V_g_profile":None, # if use the V_g_profile to determine V_w, set to True, otherwise (use the V_g_ave to determine V_w) set to False.
    "P_tot_given_profile":False,
    "P_installed": 780, # kW  
    "P_tot_given": True, # kW
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": True,
    "sailing_upstream":False,
    "wind_influence": False, # if consider wind influence, set to True; otherwise set to False.
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.9, 
    "C_year": 1961,
}             


### arrange input table

In [5]:
stretch = ["s1","s2","s3","s4","s5","s6","s7","s8",
          "s9","s10","s11","s12","s13","s14","s15"]
width = [150]
depth = [7.5, 4.18,7.83,6.5,10.6,6.45,8.04,5.65,8.87,4.93,7.7,9.8,5.45]
power_applied_down = [620,500,250]
power_applied_up = [620]

In [6]:
# prepare the work to be done by creating a list of all combinations
work_up = list(itertools.product(stretch, depth, width,power_applied_up))

# prepare a list of dictionaries for pandas
rows_up = []
for item in work_up:
    row_up = {"stretch": item[0],"depth": item[1],"width":item[2], "power_applied_up":item[3]}
    rows_up.append(row_up)

    # these are all the simulations that we want to run

# convert them to dataframe, so that we can apply a function and monitor progress
work_up_df = pd.DataFrame(rows_up)
work_up_df.tail(100)

,stretch,depth,width,power_applied_up
95,s8,10.60,150,620
96,s8,6.45,150,620
97,s8,8.04,150,620
98,s8,5.65,150,620
99,s8,8.87,150,620
...,...,...,...,...
190,s15,8.87,150,620
191,s15,4.93,150,620
192,s15,7.70,150,620
193,s15,9.80,150,620


### Run simulation


In [7]:
results_up = []

for i, row_up in tqdm.tqdm(work_up_df.iterrows(),disable=True):

    # get vessel 
    data_vessel_i = data_vessel_up.copy()
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row_up['power_applied_up']
    
    stretch = row_up['stretch']
    # estimate 'grounding speed' as a useful upperbound
    # upperbound, selected, results_df = opentnsim.strategy.get_upperbound_for_power2v(vessel, width=row['width'], depth=row['depth'], margin=0.3,bounds=(0, 20))
    # print(upperbound)
    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    V_w_up = opentnsim.strategy.power2v(vessel, h_0=row_up['depth'],power_applied=row_up['power_applied_up'], upperbound=5)
    # V_w_down = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_down'], upperbound=5)
    
    result_up ={}
    result_up.update(row_up)
    
    result_up['V_w_up (m/s)'] = V_w_up    
    result_up['V_w_up (km/h)'] = V_w_up * 3.6
    # result_up['V_w_down (m/s)'] = V_w_down    
    # result_up['V_w_down (km/h)'] = V_w_down * 3.6
    results_up.append(result_up)

In [8]:
results_up_df = pd.DataFrame(results_up)
results_up_df

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h)
0,s1,7.50,150,620,3.176066,11.433838
1,s1,4.18,150,620,3.167298,11.402274
2,s1,7.83,150,620,3.176347,11.434849
3,s1,6.50,150,620,3.174927,11.429737
4,s1,10.60,150,620,3.177885,11.440385
...,...,...,...,...,...,...
190,s15,8.87,150,620,3.177243,11.438074
191,s15,4.93,150,620,3.171276,11.416592
192,s15,7.70,150,620,3.176241,11.434466
193,s15,9.80,150,620,3.177625,11.439452


In [9]:
V1_up = results_up_df.query('stretch == "s1" & depth == 7.5 & power_applied_up == 620')
V2_up = results_up_df.query('stretch == "s2" & depth == 4.18 & power_applied_up == 620')
V3_up = results_up_df.query('stretch == "s3" & depth == 4.18 & power_applied_up == 620')
V4_up = results_up_df.query('stretch == "s4" & depth == 7.83 & power_applied_up == 620')
V5_up = results_up_df.query('stretch == "s5" & depth == 7.5 & power_applied_up == 620')
V6_up = results_up_df.query('stretch == "s6" & depth == 6.5 & power_applied_up == 620')
V7_up = results_up_df.query('stretch == "s7" & depth == 10.6 & power_applied_up == 620')
V8_up = results_up_df.query('stretch == "s8" & depth == 6.45 & power_applied_up == 620')
V9_up = results_up_df.query('stretch == "s9" & depth == 8.04 & power_applied_up == 620')
V10_up = results_up_df.query('stretch == "s10" & depth == 5.65 & power_applied_up == 620')
V11_up = results_up_df.query('stretch == "s11" & depth == 8.87 & power_applied_up == 620')
V12_up = results_up_df.query('stretch == "s12" & depth == 4.93 & power_applied_up == 620')
V13_up = results_up_df.query('stretch == "s13" & depth == 7.7 & power_applied_up == 620')
V14_up = results_up_df.query('stretch == "s14" & depth == 9.8 & power_applied_up == 620')
V15_up = results_up_df.query('stretch == "s15" & depth == 5.45 & power_applied_up == 620')

V_up = pd.concat([V1_up,V2_up,V3_up,V4_up,V5_up,V6_up,V7_up,V8_up,V9_up,V10_up,V11_up,V12_up,V13_up,V14_up,V15_up])

V_up= V_up.drop_duplicates()
V_up.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
V_up

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h)
1,s1,7.50,150,620,3.176066,11.433838
2,s2,4.18,150,620,3.167298,11.402274
3,s3,4.18,150,620,3.167298,11.402274
4,s4,7.83,150,620,3.176347,11.434849
5,s5,7.50,150,620,3.176066,11.433838
6,s6,6.50,150,620,3.174927,11.429737
7,s7,10.60,150,620,3.177885,11.440385
8,s8,6.45,150,620,3.174855,11.429477
9,s9,8.04,150,620,3.176508,11.435428
10,s10,5.65,150,620,3.173391,11.424206


In [10]:
stretch = ["s1","s2","s3","s4","s5","s6","s7","s8",
          "s9","s10","s11","s12","s13","s14","s15"]
width = [150]
# depth = [7.5, 3.7744,7.7178,6.5,10.6,6.5531,8.1001,5.8238,8.9354,4.9192,7.7,9.8,5.5799]
depth = [7.5, 4.18,7.83,6.5,10.6,6.45,8.04,5.65,8.87,4.93,7.7,9.8,5.45]
power_applied_down = [620,500,250]
power_applied_up = [620]

In [11]:
# prepare the work to be done by creating a list of all combinations
work_down = list(itertools.product(stretch, depth, width, power_applied_down))

# prepare a list of dictionaries for pandas
rows_down = []
for item in work_down:
    row_down = {"stretch": item[0], "depth": item[1],"width":item[2], "power_applied_down": item[3]}
    rows_down.append(row_down)

    # these are all the simulations that we want to run

# convert them to dataframe, so that we can apply a function and monitor progress
work_down_df = pd.DataFrame(rows_down)
work_down_df.tail(100)

,stretch,depth,width,power_applied_down
485,s13,6.45,150,250
486,s13,8.04,150,620
487,s13,8.04,150,500
488,s13,8.04,150,250
489,s13,5.65,150,620
...,...,...,...,...
580,s15,9.80,150,500
581,s15,9.80,150,250
582,s15,5.45,150,620
583,s15,5.45,150,500


In [12]:
results_down = []

for i, row_down in tqdm.tqdm(work_down_df.iterrows(),disable=True):

    # get vessel 
    data_vessel_i = data_vessel_down.copy()
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row_down['power_applied_down']
    stretch = row_down['stretch']
    # estimate 'grounding speed' as a useful upperbound
    # upperbound, selected, results_df = opentnsim.strategy.get_upperbound_for_power2v(vessel, width=row['width'], depth=row['depth'], margin=0.3,bounds=(0, 20))
    # print(upperbound)
    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    # V_w_up = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_up'], upperbound=5)
    V_w_down = opentnsim.strategy.power2v(vessel, h_0=row_down['depth'],power_applied=row_down['power_applied_down'], upperbound=5)
    
    result_down ={}
    result_down.update(row_down)
    
    # result['V_w_up (m/s)'] = V_w_up    
    # result['V_w_up (km/h)'] = V_w_up * 3.6
    result_down['V_w_down (m/s)'] = V_w_down    
    result_down['V_w_down (km/h)'] = V_w_down * 3.6
    results_down.append(result_down)

In [13]:
results_down_df = pd.DataFrame(results_down)
results_down_df

,stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h)
0,s1,7.50,150,620,3.262139,11.743701
1,s1,7.50,150,500,3.166012,11.397643
2,s1,7.50,150,250,2.860431,10.297550
3,s1,4.18,150,620,3.260024,11.736085
4,s1,4.18,150,500,3.163627,11.389056
...,...,...,...,...,...,...
580,s15,9.80,150,500,3.166560,11.399616
581,s15,9.80,150,250,2.861259,10.300533
582,s15,5.45,150,620,3.261241,11.740466
583,s15,5.45,150,500,3.164998,11.393991


In [14]:
V1_down = results_down_df.query('stretch == "s1" & depth == 7.5 & power_applied_down == 620')
V2_down = results_down_df.query('stretch == "s2" & depth == 4.18 & power_applied_down == 620')
V3_down = results_down_df.query('stretch == "s3" & depth == 4.18 & power_applied_down == 620')
V4_down = results_down_df.query('stretch == "s4" & depth == 7.83 & power_applied_down == 620')
V5_down = results_down_df.query('stretch == "s5" & depth == 7.5 & power_applied_down == 620')
V6_down = results_down_df.query('stretch == "s6" & depth == 6.5 & power_applied_down == 620')
V7_down = results_down_df.query('stretch == "s7" & depth == 10.6 & power_applied_down == 620')
V8_down = results_down_df.query('stretch == "s8" & depth == 6.45 & power_applied_down == 620')
V9_down = results_down_df.query('stretch == "s9" & depth == 8.04 & power_applied_down == 620')
V10_down = results_down_df.query('stretch == "s10" & depth == 5.65 & power_applied_down == 620')
V11_down = results_down_df.query('stretch == "s11" & depth == 8.87 & power_applied_down == 620')
V12_down = results_down_df.query('stretch == "s12" & depth == 4.93 & power_applied_down == 500')
V13_down = results_down_df.query('stretch == "s13" & depth == 7.7 & power_applied_down == 620')
V14_down = results_down_df.query('stretch == "s14" & depth == 9.8 & power_applied_down == 250')
V15_down = results_down_df.query('stretch == "s15" & depth == 5.45 & power_applied_down == 620')
V_down = pd.concat([V1_down,V2_down,V3_down,V4_down,V5_down,V6_down,
                    V7_down,V8_down,V9_down,V10_down,V11_down,V12_down,V13_down,V14_down,V15_down])
V_down= V_down.drop_duplicates()
V_down.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
V_down

,stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h)
1,s1,7.50,150,620,3.262139,11.743701
2,s2,4.18,150,620,3.260024,11.736085
3,s3,4.18,150,620,3.260024,11.736085
4,s4,7.83,150,620,3.262230,11.744028
5,s5,7.50,150,620,3.262139,11.743701
6,s6,6.50,150,620,3.261791,11.742448
7,s7,10.60,150,620,3.262735,11.745846
8,s8,6.45,150,620,3.261770,11.742372
9,s9,8.04,150,620,3.262283,11.744220
10,s10,5.65,150,620,3.261366,11.740919


### get vessel velocity to the ground: upstream, downstream

In [15]:
current_speeds = pd.DataFrame([-2.24,-3.4,-4.93,-2.59,-3.26,-3.85,-1.87,-1.08,-2.83,-4.99,-2.89,-6.4,-3.42,-1.64,-4.1])
current_speeds.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
current_speeds.columns =['current_speeds (km/h)']

stretch_length = pd.DataFrame([48.28,46.02,32.23,18.87,27.4,19.2,37,15,94,30,87,48,21,37.56,171])
stretch_length.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
stretch_length.columns =['stretch_length (km)']

V_df = pd.concat([stretch_length,current_speeds,V_up,V_down], axis = 1)
V_df['V_g_upstream (km/h)']= V_df['V_w_up (km/h)']+V_df['current_speeds (km/h)']
V_df['V_g_downstream (km/h)']= V_df['V_w_down (km/h)']-V_df['current_speeds (km/h)']
V_df['duration_upstream (h)']= V_df['stretch_length (km)']/V_df['V_g_upstream (km/h)']
V_df['duration_downstream (h)']= V_df['stretch_length (km)']/V_df['V_g_downstream (km/h)']

V_df

,stretch_length (km),current_speeds (km/h),stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h),stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h),V_g_upstream (km/h),V_g_downstream (km/h),duration_upstream (h),duration_downstream (h)
1,48.28,-2.24,s1,7.50,150,620,3.176066,11.433838,s1,7.50,150,620,3.262139,11.743701,9.193838,13.983701,5.251344,3.452591
2,46.02,-3.40,s2,4.18,150,620,3.167298,11.402274,s2,4.18,150,620,3.260024,11.736085,8.002274,15.136085,5.750865,3.040416
3,32.23,-4.93,s3,4.18,150,620,3.167298,11.402274,s3,4.18,150,620,3.260024,11.736085,6.472274,16.666085,4.979702,1.933867
4,18.87,-2.59,s4,7.83,150,620,3.176347,11.434849,s4,7.83,150,620,3.262230,11.744028,8.844849,14.334028,2.133445,1.316448
5,27.40,-3.26,s5,7.50,150,620,3.176066,11.433838,s5,7.50,150,620,3.262139,11.743701,8.173838,15.003701,3.352159,1.826216
6,19.20,-3.85,s6,6.50,150,620,3.174927,11.429737,s6,6.50,150,620,3.261791,11.742448,7.579737,15.592448,2.533070,1.231365
7,37.00,-1.87,s7,10.60,150,620,3.177885,11.440385,s7,10.60,150,620,3.262735,11.745846,9.570385,13.615846,3.866093,2.717422
8,15.00,-1.08,s8,6.45,150,620,3.174855,11.429477,s8,6.45,150,620,3.261770,11.742372,10.349477,12.822372,1.449349,1.169830
9,94.00,-2.83,s9,8.04,150,620,3.176508,11.435428,s9,8.04,150,620,3.262283,11.744220,8.605428,14.574220,10.923338,6.449745
10,30.00,-4.99,s10,5.65,150,620,3.173391,11.424206,s10,5.65,150,620,3.261366,11.740919,6.434206,16.730919,4.662580,1.793087


### get vessel sailing duration: upstream, downstream, roundtrip

In [16]:
upstream_total_duration = V_df['duration_upstream (h)'].sum()
upstream_total_duration

94.45388739543625

In [17]:
downstream_total_duration = V_df['duration_downstream (h)'].sum()
downstream_total_duration

48.89906935380415